## Workflow to select modelling data (inspired from Lenselink et _al._ J Cheminform 2017 9:45)
#### NB: requires a local installation of ChEMBL (Oracle here). Works with ChEMBL_23 but should work witht the following releases.

In [153]:
from sqlalchemy import create_engine

import pandas as pd
import numpy as np

from utils import selected_prot_each_step, usmiles

In [3]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000

## Database connexion

In [4]:
# Personal login details in correct format for sqlalchemy

with open('/Users/nbosc/notebooks/alchemy_nbosc_login.txt', 'r') as f:
    engine = create_engine(f.read())

## Collect the bioactivities containing actives<br>
- From the scientific literature
- From the pharmalogical set of DruxMatrix


In [13]:
sql = '''
-- Literature data
SELECT a.doc_id
    , h.year
    , a.pchembl_value
    , e.canonical_smiles
    , c.chembl_id as mol_chemblid
    , g.chembl_id as target_chemblid
    , g.pref_name
    , g.target_type
FROM
  CHEMBL_23.ACTIVITIES a JOIN CHEMBL_23.MOLECULE_HIERARCHY b ON a.MOLREGNO = b.MOLREGNO
  JOIN CHEMBL_23.MOLECULE_DICTIONARY c ON b.PARENT_MOLREGNO = c.MOLREGNO
  JOIN CHEMBL_23.COMPOUND_PROPERTIES d ON b.PARENT_MOLREGNO = d.MOLREGNO
  JOIN CHEMBL_23.COMPOUND_STRUCTURES e ON b.PARENT_MOLREGNO = e.MOLREGNO
  JOIN CHEMBL_23.ASSAYS f ON a.ASSAY_ID = f.ASSAY_ID
  JOIN CHEMBL_23.TARGET_DICTIONARY g ON f.TID = g.TID
  JOIN CHEMBL_23.DOCS h ON a.DOC_ID = h.DOC_ID
WHERE
   a.pchembl_value IS NOT NULL
  AND (a.potential_duplicate = 0 OR a.potential_duplicate IS NULL)
  AND a.data_validity_comment IS NULL
  AND (
  (f.confidence_score = 9 AND g.target_type = 'SINGLE PROTEIN')
  OR
  (f.confidence_score = 7 AND g.target_type = 'PROTEIN COMPLEX')
  )
  AND f.src_id = 1
  AND (NOT (a.activity_comment LIKE 'inconclusive' OR a.activity_comment LIKE 'undetermined')
      OR a.activity_comment IS NULL)
  AND d.psa IS NOT NULL -- if this descriptor cannot be calculated, it is usually a signal that the molecule fill fail somehow later

UNION ALL
-- DrugMatrix data
SELECT a.doc_id
    , h.year
    , case when a.pchembl_value is null then 4.0 else a.pchembl_value end
    , e.canonical_smiles
    , c.chembl_id as mol_chemblid
    , g.chembl_id as target_chemblid
    , g.pref_name
    , g.target_type
FROM
  CHEMBL_23.ACTIVITIES a JOIN CHEMBL_23.MOLECULE_HIERARCHY b ON a.MOLREGNO = b.MOLREGNO
  JOIN CHEMBL_23.MOLECULE_DICTIONARY c ON b.PARENT_MOLREGNO = c.MOLREGNO
  JOIN CHEMBL_23.COMPOUND_PROPERTIES d ON b.PARENT_MOLREGNO = d.MOLREGNO
  JOIN CHEMBL_23.COMPOUND_STRUCTURES e ON b.PARENT_MOLREGNO = e.MOLREGNO
  JOIN CHEMBL_23.ASSAYS f ON a.ASSAY_ID = f.ASSAY_ID
  JOIN CHEMBL_23.TARGET_DICTIONARY g ON f.TID = g.TID
  JOIN CHEMBL_23.DOCS h ON a.DOC_ID = h.DOC_ID
WHERE
    f.src_id = 15 -- drugmatrix
    AND g.target_type IN ('SINGLE PROTEIN', 'PROTEIN COMPLEX')
    AND a.standard_type = 'IC50'
    AND d.PSA IS NOT NULL
  '''
df = pd.read_sql(sql, engine)

In [14]:
df.shape

(656263, 8)

### Split activities and molecule structures 

#### Molecules 

In [15]:
df_mols = df[['mol_chemblid', 'canonical_smiles']].drop_duplicates()

#### Activities 

In [16]:
df_act = df[['doc_id', 'year', 'pchembl_value', 'mol_chemblid', 'target_chemblid', 'pref_name', 'target_type']]

### Generate stereo free smiles (USMILES) <br>
Stereochenistry is an aspect of the molecule very important. Indeed, for a given chiral molecule, one stereoisomer might be a potent inhibitor when the other one is completly inactive.<br>
However, the morgan fingerprint does not take into account this property of the molecules. A flag can be activated but a quick analysis shows that the resulting similarity measurements are odd<br>
Therefore, in this work, two stereoisomers are considered as the same molecule

In [17]:
df_mols, failed1, failed2 = usmiles(df_mols, 'canonical_smiles')

Molecules RDKIT cannot proceed: 2
Molcules to generate usmiles: 293021
Fail to generate usmiles: 0


## Remove activity duplicates<br>
### If several datapoints associated to pair protein-ligand, keep the median value and remove duplicates 

In [18]:
df_act.shape

(656263, 7)

#### Get back the usmiles. It acts as unique identifier for molecules 

In [19]:
df_act = df_act.merge(df_mols[['mol_chemblid', 'usmiles']], on = 'mol_chemblid')

#### Need to define a year for DrugMatrix data other groupby will get rid of these data 

In [20]:
df_act = df_act.fillna(2011.0)

#### Calculate the median value for pchembls. For doc_id (will be used for diversity) and year (will be used for temporal split) we keep the first occurence and the oldest year respectively 

In [21]:
def fun(x):
    names = {
        'pchembl_median': x['pchembl_value'].median()
        , 'doc_id': x.iloc[0]['doc_id']
        , 'year': x['year'].min()
    }
    return(pd.Series(names, index = ['pchembl_median', 'doc_id', 'year']))
df_act_u = df_act.groupby(['target_chemblid', 'pref_name', 'target_type', 'usmiles']).apply(fun).reset_index()

## The active/inactive split is done using [IDG](https://druggablegenome.net/ProteinFam) thresholds which distinguish few protein families

### IDG thresholds<br>
Active /inactive categorisation is based on IDG activity threshold:

- Kinases: <= 30nM (>= 7.5 pchembl_value)
- GPCRs: <= 100nM (>= 7 pchembl_value)
- Nuclear Receptors: <= 100nM (>= 7 pchembl_value)
- Ion Channels: <= 10μM (>= 5 pchembl_value)
- Non-IDG Family Targets: <= 1μM (>= 6 pchembl_value)


In [22]:
target_active_threshold = pd.DataFrame({'activity_threshold': [7.5, 7, 7, 7, 7, 5, 5, 5, 6], 'target_class': ['Enzyme > Kinase > Protein Kinase'
                                                                                                ,'Membrane receptor > Family A G protein-coupled receptor'
                                                                                                ,'Membrane receptor > Family B G protein-coupled receptor'
                                                                                                ,'Membrane receptor > Family C G protein-coupled receptor'
                                                                                                ,'Transcription factor > Nuclear receptor'
                                                                                                ,'Ion channel > Ligand-gated ion channel'
                                                                                                ,'Ion channel > Voltage-gated ion channel'
                                                                                                ,'Ion channel > Other ion channel'
                                                                                                ,'Others']})

### Load fixed target classification
List was adapted from ChEMBL target classification to make it simpler and keep one family per target

In [23]:
df_class = pd.read_excel('target_class_fixed.xls')

#### Get the target class from each protein with activities 

In [24]:
df_act_unique = df_act_u.merge(df_class[['target_chemblid', 'target_class', 'organism']], on = 'target_chemblid')

#### Few protein are lost at this stage because they don't have a target class

#### Merge the activities with the thresholds
#### Protein families with no specific threshold have a default threshold 

In [25]:
df_act_unique = df_act_unique.merge(target_active_threshold, on='target_class', how='left').fillna(float(target_active_threshold.loc[target_active_threshold.target_class == 'Others']['activity_threshold']))

### Distinguish active/inactive by comparing the median of the pchembl value with the threshold 

In [26]:
df_act_unique['activity_class'] = np.where(df_act_unique['pchembl_median'] >= df_act_unique['activity_threshold'], 'active', 'inactive')

## Now actives are precisely defined, select only targets with enough data for modelling
### Remove targets if they have less than 40 actives

In [27]:
dfm2 = pd.DataFrame(df_act_unique.groupby(['target_chemblid', 'activity_class'])['usmiles'].nunique()).reset_index()

dfm3 = dfm2[(dfm2['usmiles'] >= 40) & (dfm2['activity_class']=='active')].reset_index()

In [29]:
df_act_unique2 = df_act_unique.loc[df_act_unique.target_chemblid.isin(dfm3.target_chemblid)] 

### Keep targets if tested in at least 2 documents

In [31]:
dfl3 = pd.DataFrame(df_act_unique2.groupby(['target_chemblid'])['doc_id'].nunique())
dfl4 = dfl3[dfl3['doc_id'] >= 2].reset_index()

In [32]:
df_act_unique3 = df_act_unique2.loc[df_act_unique2.target_chemblid.isin(dfl4.target_chemblid)]

#### Count how many actives/inactives per targets 

In [33]:
df_act_unique3_count = df_act_unique3.groupby(['target_chemblid', 'target_class', 'activity_class'])['usmiles'].count().reset_index()

In [34]:
df_act_unique3_count.target_chemblid.nunique()

834

---

## Collect more inactives
Inactives will be identified starting from a set of records in which a '>' relation exists and by only considering the activity types used for the pchembl calculation. All the other filters are the same than used previously

In [51]:
sql = '''
SELECT a.doc_id
    , h.year
    , a.standard_value
    , e.canonical_smiles
    , c.chembl_id as mol_chemblid
    , g.chembl_id as target_chemblid
    , g.pref_name
    , g.target_type
FROM
  CHEMBL_23.ACTIVITIES a JOIN CHEMBL_23.MOLECULE_HIERARCHY b ON a.MOLREGNO = b.MOLREGNO
  JOIN CHEMBL_23.MOLECULE_DICTIONARY c ON b.PARENT_MOLREGNO = c.MOLREGNO
  JOIN CHEMBL_23.COMPOUND_PROPERTIES d ON b.PARENT_MOLREGNO = d.MOLREGNO
  JOIN CHEMBL_23.COMPOUND_STRUCTURES e ON b.PARENT_MOLREGNO = e.MOLREGNO
  JOIN CHEMBL_23.ASSAYS f ON a.ASSAY_ID = f.ASSAY_ID
  JOIN CHEMBL_23.TARGET_DICTIONARY g ON f.TID = g.TID
  JOIN CHEMBL_23.DOCS h ON a.DOC_ID = h.DOC_ID
WHERE
  g.chembl_id IN {} -- consider only inactive compounds for proteins already selected
  AND a.PCHEMBL_VALUE IS null
  AND a.standard_value IS NOT null
  AND a.standard_type IN ('IC50', 'EC50', 'XC50', 'AC50', 'Ki', 'Kd', 'Potency') -- same as for pchembl_value
  AND a.standard_units = 'nM'
  AND a.standard_relation = '>'
  AND (a.POTENTIAL_DUPLICATE = 0 OR a.POTENTIAL_DUPLICATE IS NULL)
  AND a.DATA_VALIDITY_COMMENT IS NULL
  AND (
  (f.CONFIDENCE_SCORE = 9 AND g.TARGET_TYPE = 'SINGLE PROTEIN')
  OR
  (f.CONFIDENCE_SCORE = 7 AND g.TARGET_TYPE = 'PROTEIN COMPLEX')
  )
  AND f.src_id = 1
  AND (NOT (a.ACTIVITY_COMMENT LIKE 'inconclusive' OR a.activity_comment LIKE 'undetermined')
      OR a.ACTIVITY_COMMENT IS NULL)
  AND d.PSA IS NOT NULL -- if this descriptor cannot be calculated, it is usually a signal that the molecule does not suit for fingerprint calculation
'''

df_inactives = pd.read_sql(sql.format(tuple(df_act_unique3.target_chemblid.unique())), engine)

#### Easier to work with a pchembl like value 

In [55]:
df_inactives['pchembl_value'] = df_inactives.standard_value.apply(lambda x: -np.log10(x*np.power(10, -9.0)))

#### Keep the lowest pchembl-value

In [57]:
df_inactives = df_inactives.sort_values('pchembl_value', ascending = True)
df_inactives = df_inactives.drop_duplicates(subset=['mol_chemblid', 'target_chemblid'], keep='first')

### Split activities and molecule structures 

In [58]:
df_mol_inactive = df_inactives[['mol_chemblid', 'canonical_smiles']].drop_duplicates()

In [71]:
df_act_inactive = df_inactives[['doc_id', 'year', 'pchembl_value', 'mol_chemblid', 'target_chemblid', 'pref_name', 'target_type']]

### Generate stereo free smiles (USMILES) <br>
Stereochenistry is an aspect of the molecule very important. Indeed, for a given chiral molecule, one stereoisomer might be a potent inhibitor when the other one is completly inactive.<br>
However, the morgan fingerprint does not take into account this property of the molecules. A flag can be activated but a quick analysis shows that the results are not correct.<br>
Therefore, for this work, two stereoisomers are considered as the same molecule

In [72]:
df_mol_inactive, failed_inactive1, failed_inactive2 = usmiles(df_mol_inactive, 'canonical_smiles')

Molecules RDKIT cannot proceed: 0
Molcules to generate usmiles: 46244
Fail to generate usmiles: 0


### Distinguish really inactive compounds using IDG thresholds<br>

In [73]:
df_act_inactive.shape

(72579, 7)

In [74]:
df_act_inactive = df_act_inactive.merge(df_mol_inactive[['mol_chemblid', 'usmiles']], on = 'mol_chemblid')

In [77]:
df_act_inactive = df_act_inactive.merge(df_class[['target_chemblid', 'target_class', 'organism']], on = 'target_chemblid')

In [80]:
df_act_inactive = df_act_inactive.merge(target_active_threshold, on='target_class', how='left' ).fillna(float(target_active_threshold.loc[target_active_threshold.target_class == 'Others']['activity_threshold']))

In [81]:
df_act_inactive['activity_class'] = np.where(df_act_inactive['pchembl_value'] >= df_act_inactive['activity_threshold'], 'active', 'inactive')

#### Few activities are classified actives but don't forget because of the '>' relations, they cannot be considered as active here

In [82]:
df_act_inactive.loc[df_act_inactive.activity_class == 'active'].shape

(5326, 12)

In [83]:
df_act_inactive.loc[df_act_inactive.activity_class == 'inactive'].shape

(67253, 12)

In [84]:
df_act_inactive_sure = df_act_inactive.loc[df_act_inactive.activity_class == 'inactive']

In [86]:
df_act_inactive_kept = df_act_inactive_sure.loc[df_act_inactive_sure.target_chemblid.isin(df_act_unique3.target_chemblid)]

## Now merge active/inactive set with inactive set<br>
For the active/inactive set, we consider only the version where there are enough actives.
See it helps collecting more inactives

In [89]:
df_merged = pd.concat([df_act_unique3[['target_chemblid', 'pref_name', 'usmiles', 'doc_id', 'year', 'target_class', 'target_type', 'organism', 'activity_class', 'activity_threshold']], df_act_inactive_kept[['target_chemblid', 'pref_name', 'usmiles', 'doc_id', 'year', 'target_class', 'target_type', 'organism', 'activity_class', 'activity_threshold']]]).sort_values('activity_class')

#### Drop duplicates keeping the first occurence
That way, if a protein-molecule pair is both classified active and inactive, only keep active which was determined more precisely (pchembl_value vs '>' relation) 

In [91]:
df_merged = df_merged.drop_duplicates(['target_chemblid', 'usmiles'], keep = 'first')

#### Check that we still have the same number of proteins with enough actives for modelling.

In [93]:
df_merged_count = df_merged.groupby(['target_chemblid', 'activity_class', 'target_class'])['usmiles'].count().reset_index()

### For inactives, let's consider it 30 of them are enough for modelling
### Therefore, each QSAR model will be built with at least 70 disctincts compounds (40 actives + 30 inactives)

In [97]:
df_merged = df_merged.loc[df_merged.target_chemblid.isin(df_merged_count.loc[(df_merged_count.activity_class == 'inactive') & (df_merged_count.usmiles >= 30)]['target_chemblid'].unique())]

---

## For protein that could not match the selection criteria, let's see if we can use a 6.5 activity threshold  to get them back

#### Identifiy these proteins

In [98]:
df_not_selected = df_act_unique.loc[~df_act_unique.target_chemblid.isin(df_merged.target_chemblid)].drop(['activity_threshold','activity_class'], axis = 1)

In [101]:
df_not_selected['activity_threshold'] = 6.5

In [102]:
df_not_selected['activity_class'] = np.where(df_not_selected['pchembl_median'] >= df_not_selected['activity_threshold'], 'active', 'inactive')

### Reperform the selection on this subset

In [104]:
### Remove targets if they have less than 40 actives

dfm2 = pd.DataFrame(df_not_selected.groupby(['target_chemblid', 'activity_class'])['usmiles'].nunique()).reset_index()

dfm3 = dfm2[(dfm2['usmiles'] >= 40) & (dfm2['activity_class']=='active')].reset_index()

df_not_selected2 = df_not_selected.loc[df_not_selected.target_chemblid.isin(dfm3.target_chemblid)] 

### Keep targets if tested in at least 2 documents

dfl3 = pd.DataFrame(df_not_selected2.groupby(['target_chemblid'])['doc_id'].nunique())
dfl4 = dfl3[dfl3['doc_id'] >= 2].reset_index()

df_not_selected3 = df_not_selected2.loc[df_not_selected2.target_chemblid.isin(dfl4.target_chemblid)]


#### Count how many actives/inactives per targets 

df_not_selected3_count = df_not_selected3.groupby(['target_chemblid', 'target_class', 'activity_class'])['usmiles'].count().reset_index()

### Add the inactive set for these proteins and see if more can be selected 

In [106]:
df_inactive_not_selected = df_act_inactive.loc[df_act_inactive.target_chemblid.isin(df_not_selected3.target_chemblid)].drop(['activity_threshold','activity_class'], axis = 1)

In [109]:
df_inactive_not_selected['activity_threshold'] = 6.5

df_inactive_not_selected['activity_class'] = np.where(df_inactive_not_selected['pchembl_value'] >= df_inactive_not_selected['activity_threshold'], 'active', 'inactive')

In [110]:
df_act_inactive_sure2 = df_inactive_not_selected.loc[df_inactive_not_selected.activity_class == 'inactive']

### Merge the two previous sets

In [112]:
df_not_selected_merged = pd.concat([df_not_selected3[['target_chemblid', 'pref_name', 'target_type', 'organism', 'usmiles', 'doc_id', 'year', 'target_class', 'activity_class', 'activity_threshold']], df_act_inactive_sure2[['target_chemblid', 'pref_name', 'target_type', 'organism', 'usmiles', 'doc_id', 'year', 'target_class', 'activity_class', 'activity_threshold']]]).sort_values('activity_class')

#### Drop duplicates keeping the first occurence
That way, if a protein-molecule pair is both classified active and inactive, only keep active which was determined more precisely (pchembl_value vs '>' relation) 

In [114]:
df_not_selected_merged = df_not_selected_merged.drop_duplicates(['target_chemblid', 'usmiles'], keep = 'first')

#### Check that we still have the same number of proteins with enough actives for modelling.

In [116]:
df_not_selected_merged_count = df_not_selected_merged.groupby(['target_chemblid', 'activity_class', 'target_class'])['usmiles'].count().reset_index()

### For inactives, let's consider it 30 of them are enough for modelling
### Therefore, each QSAR model will be built with at least 70 disctincts compounds (40 actives + 30 inactives)

In [119]:
df_not_selected_merged = df_not_selected_merged.loc[df_not_selected_merged.target_chemblid.isin(df_not_selected_merged_count.loc[(df_not_selected_merged_count.activity_class == 'inactive') & (df_not_selected_merged_count.usmiles >= 30)]['target_chemblid'].unique())]

---

## Finally, merge the result of the two selection 

In [120]:
#### Do some sanity check
print(df_merged.loc[df_merged.target_chemblid.isin(df_not_selected_merged.target_chemblid.unique())].shape)
print(df_not_selected_merged.loc[df_not_selected_merged.target_chemblid.isin(df_merged.target_chemblid.unique())].shape)

(0, 10)
(0, 10)


In [121]:
df_selection_final = pd.concat([df_merged, df_not_selected_merged])

/Users/nbosc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


## Summary 

### Check threshold effect on protein class numbers

In [142]:
summary = selected_prot_each_step(df_act_unique, df_act_unique2, df_act_unique3, df_merged, df_selection_final, target_active_threshold)
summary

,target_class,before_selection,activity_threshold,40_actives,2_documents,40_actives_30_inactives,final_selection,fraction_of_proteins_not_selected,protein_complexes
0,Enzyme > Kinase > Protein Kinase,519,7.5,79,79,77,117,0.774566,9
1,Membrane receptor > Family A G protein-coupled receptor,494,7.0,186,186,176,195,0.605263,0
2,Membrane receptor > Family B G protein-coupled receptor,24,7.0,5,5,4,4,0.833333,0
3,Membrane receptor > Family C G protein-coupled receptor,25,7.0,8,8,8,9,0.640000,0
4,Transcription factor > Nuclear receptor,91,7.0,21,21,21,25,0.725275,0
5,Ion channel > Ligand-gated ion channel,138,5.0,47,47,20,35,0.746377,19
6,Ion channel > Voltage-gated ion channel,98,5.0,34,33,13,20,0.795918,0
7,Ion channel > Other ion channel,11,5.0,5,5,5,5,0.545455,0
8,Others,2643,6.0,456,450,351,378,0.856981,19
9,Sum,4043,NaN,841,834,675,788,0.805095,47


In [143]:
summary.to_excel('protein_selection_summary.xlsx')

### Some observations 

## Export the activity set

In [144]:
df_selection_final.to_excel('dataset_ready.xlsx')

## Create a file to summarise the target information

In [145]:
tmp = df_selection_final.groupby(['target_chemblid', 'activity_class'])['doc_id'].count().reset_index().sort_values('doc_id').pivot(index='target_chemblid', columns='activity_class', values='doc_id').reset_index()
df_selection_final[['target_chemblid', 'pref_name', 'target_class', 'organism', 'target_type', 'activity_threshold']].drop_duplicates().merge(tmp, on='target_chemblid').to_excel('modelled_protein_info.xlsx')

## Finalise and export the set of compounds<br>
Include molecule from the inactive set that are not already in the molecule set

In [146]:
df_mols_kept = df_mols.loc[df_mols.usmiles.isin(df_selection_final.usmiles)]

In [147]:
df_mol_inactive_kept = df_mol_inactive.loc[df_mol_inactive.usmiles.isin(df_selection_final.usmiles)]

In [148]:
df_mols_all_kept = pd.concat([df_mols_kept, df_mol_inactive_kept])

In [149]:
df_mols_all_kept.shape

(304405, 4)

#### Group molecules using stereo free SMILES

In [150]:
def func(x):
    names = {
        'num_mols': x['mol_chemblid'].nunique()
        , 'mol_ids': ', '.join(set(x['mol_chemblid']))
    }
    return(pd.Series(names, index = ['num_mols', 'mol_ids']))

df_mols_all_kept_unique = df_mols_all_kept.groupby(['usmiles']).apply(func).reset_index()

In [151]:
df_mols_all_kept_unique.to_excel('mol_inchi_for_modelling.xlsx')